In [1]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter3D
import src.simulator.CustomManeuvres as CustomManeuvres
import copy

In [8]:
# Define the two coplanar debris
a = 7245.172213925252 << u.km
ecc = 0 << u.one
inc = 9.50714306409916 << u.deg
raan = 7.319939418114051  << u.deg
argp = 0 << u.deg
nu = 35.517054310933176  << u.deg


# 7245.172213925252 km 0.0 9.50714306409916 deg 7.319939418114051 deg 0.0 deg 35.517054310933176 deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 7391.925971590665 << u.km
ecc = 0 << u.one
inc = 7.851759613930136 << u.deg
raan = 1.9967378215835974 << u.deg
argp = 0 << u.deg
nu = 5.124397828900178 << u.deg

# 7391.925971590665 km 0.0 7.851759613930136 deg 1.9967378215835974 deg 0.0 deg 5.124397828900178 deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 6477.943422637125 << u.km
ecc = 0 << u.one
inc = 14.924145688620425 << u.deg
raan = 1.50412719997725 << u.deg
argp = 0 << u.deg
nu = 70.66915465609816  << u.deg

deb_3 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
# frame.plot(deb_3)
frame.plot(otv)

In [43]:
print(otv.a, otv.ecc, otv.inc, otv.raan, otv.argp, otv.nu)

6477.944244775886 km 0.0007801881910308949 0.2607944174878461 rad 0.00993905587002115 rad 5.313941579392749 rad 1.573096502828908 rad


In [9]:
# INCLINATION CHANGE

# import src.simulator.CustomManeuvres as CustomManeuvres

inc_change = CustomManeuvres.simple_inc_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)
deb_3 = deb_3.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# PRINTOUT
print('DV USED: ', inc_change.get_total_cost())
print('DT USED: ', inc_change.get_total_time())

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(deb_3)
frame.plot(otv)

otv inc 7.851759614235339 deg
deb_2 inc 7.851759613930136 deg
DV USED:  0.21429172594682616 km / s
DT USED:  2463.17422019182 s


In [10]:
# import src.simulator.CustomManeuvres as CustomManeuvres
raan_change = CustomManeuvres.simple_raan_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = raan_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)
deb_3 = deb_3.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(raan_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# PRINTOUT
print('DV USED: ', raan_change.get_total_cost())
print('DT USED: ', raan_change.get_total_time())
print('COMBINED DV USED: ', raan_change.get_total_cost() + inc_change.get_total_cost() )

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(deb_3)
frame.plot(otv)


otv inc 8.57803275909084 deg
deb_2 inc 7.851759613930136 deg
DV USED:  0.09411092331465636 km / s
DT USED:  6092.475920470063 s
COMBINED DV USED:  0.30840264926148253 km / s


In [11]:
# Hohmann Transfer

# Calculate a basic homnann
# import src.simulator.CustomManeuvres as CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)
deb_3 = deb_3.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# PRINTOUT
print('DV USED: ', hoh_phas.get_total_cost())
print('DT USED: ', hoh_phas.get_total_time())

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(deb_3)
frame.plot(otv)
frame.plot(transfer_orbit)

DV USED:  0.07399578962877332 km / s
DT USED:  182531.32847112164 s


In [6]:
# Total resources used
total_dv = hoh_phas.get_total_cost() + raan_change.get_total_cost() + inc_change.get_total_cost()
total_time = hoh_phas.get_total_time() + raan_change.get_total_time() + inc_change.get_total_time()

# Printout
print('Total DV: ', total_dv)
print('Total Time: ', total_time)

Total DV:  0.6314069323077871 km / s
Total Time:  30680.37813218773 s


In [12]:
print('otv rv: ', otv.r)
print('deb_2 rv: ', deb_2.r)
print('deb_1 rv: ', deb_1.r)
print('distance between otv and deb_2: ', otv.r - deb_2.r)


otv rv:  [ 184.74934462 7308.69787537 1090.58361242] km
deb_2 rv:  [ 833.16418447 7276.58566973  998.85334372] km
deb_1 rv:  [-163.12248979 7144.86754387 1190.28536215] km
distance between otv and deb_2:  [-648.41483985   32.11220564   91.73026869] km
